Project by Jeremy Bouhi & Lucas Trevalinet

# Classification Document


## Imports

In [1]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
from pathlib import Path
import re
import pyprind

## STEP 1 : Grab Data

In [3]:
# We consider you had already unzip the dataset file and that this file is in the parent folder

#GLOBAL_PATH = os.path.join("Freemium_cass_global_20180315-170000","20180315-170000", "juri","cass","global") #Lucas' Path
GLOBAL_PATH = os.path.join("..","20180315-170000", "juri","cass","global") #Jeremy's Path

# We need to do that for excluding data from Juri_path
CIVILE_PATH = os.path.join(GLOBAL_PATH,"civile")
COMMERCIALE_PATH = os.path.join(GLOBAL_PATH,"commerciale")
CRIMINELLE_PATH = os.path.join(GLOBAL_PATH,"criminelle")
SOCIALE_PATH = os.path.join(GLOBAL_PATH,"sociale")

In [4]:
from nltk.tokenize import sent_tokenize

ident = []
text = []
division = []
#DATA_PATH = [SOCIALE_PATH] # For faster test
DATA_PATH = [CIVILE_PATH, COMMERCIALE_PATH, CRIMINELLE_PATH, SOCIALE_PATH]

for DIVISION_PATH in DATA_PATH :
    xml_files = list(Path(DIVISION_PATH).glob('**/*.xml'))
    
    for xml_file in xml_files:
    
        with open(xml_file, 'r', encoding="utf-8") as content:

            etree = ET.parse(content) #create an ElementTree object 
            root = etree.getroot()
            
            # For getting the ID
            for child in root.iter('META_COMMUN'):
                id = child.find('ID').text
                ident.append(id)

            
            for child in root.iter('BLOC_TEXTUEL'):
                contenu = "".join(child.itertext())
                text.append(contenu)
                
            # For getting the division    
            for child in root.iter('META_JURI_JUDI'):
                formation = re.sub('CHAMBRE|_|[0-9]', '', child.find('FORMATION').text)
                division.append(formation)


## STEP 2 : Create DataFrame to manipulate easily data

In [21]:
d = {'id': ident, 'text': text, 'division': division}
df = pd.DataFrame(data = d)
    
df[df['text']!=""]
    
# remove all NaN values from df
df.dropna()

# save for the next time
df.to_pickle('3FD8KA7.pkl')
    

In [20]:
print(df[df['text']==""])

Empty DataFrame
Columns: [id, text, division]
Index: []


In [2]:
df = pd.read_pickle('3FD8KA7.pkl')

In [321]:
fast_execution = True #to run code faster
mini_df = df.sample(500)

if(fast_execution) : 
    X = mini_df['text']
    y = mini_df['division']
    print('You chose to run using fast_execution')
    print('data size: ', X.shape[0])

else :
    X = df['text']
    y = df['division']
    print('You chose to run with the whole dataset')
    print('data size: ', X.shape[0])
    
formatted_X = X

You chose to run using fast_execution
data size:  500


## STEP 4 : Clean Data

In [322]:
def clean_references_to_law_codes(text) : 
    return re.sub('(?<=Code )civil|de l\'action sociale et des familles|de l\'artisanat|des assurances|de l\'aviation civile|du cinéma et de l image animée|de commerce|des communes( de la Nouvelle-Calédonie)?|de la consommation', '', ''.join(text))

formatted_X = formatted_X.apply(clean_references_to_law_codes)

In [323]:
# Clean references to law articles and all numbers

def clean_references_to_law_articles(text) :
    re.sub(r'((?:\b[A-Z][\.-]? ?)?(?:\d+-?\d+\b))', 'N', ''.join(text))
    text = re.sub(r'\[[0-9]*\]', '', text) #remove digits
    return text

formatted_X = formatted_X.apply(clean_references_to_law_articles)

In [324]:
# Preprocessing step pull off all the htlm tag, "\n", and any special characters
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # remove HTML tags
    text = re.sub('[,\/#!$%\^&\*:{}=\_`~()«»°–]','', text) # remove special characters except ; and . for spliting in sentences
    text = re.sub('\.\.\.','', text) # remove ...
    text = re.sub(r'\bM\. \b','M ', text) # explicit replacement of M. (because of the .)
    text = text.replace(';','.') # some sentences are split by ;
    text = text.replace('\n','').replace('\t','').replace('\'',' ')
    text.replace('\s{2,}',' ')
    return text

X = X.apply(preprocessor) # Used for a better display of the most influential sentences
formatted_X = formatted_X.apply(preprocessor)

In [325]:
# Set to lower case
def to_lower_case(s) :
    return s.lower()

formatted_X = formatted_X.apply(to_lower_case)

In [326]:
# We decided to replace any corresponding labels to a unique special label : *_* 
def clean_words_corresponding_to_labels(text) :
    return re.sub('\b(criminel(le)?|commercial(e)?|social(e)?|civil(e))\b', '*_*', ''.join(text))

formatted_X = formatted_X.apply(clean_words_corresponding_to_labels)

In [327]:
#Stop_words :
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def remove_stop_words(text):
    
    french_stopwords = set(stopwords.words('french'))
    tokens = word_tokenize(text, language='french')
    
    content_tokens = ""
    for token in tokens:
        if token not in french_stopwords:
            content_tokens += token
            content_tokens += " "
    return(content_tokens) 

formatted_X = formatted_X.apply(remove_stop_words)

In [49]:
text1 = "coucou c'est moi les amis je suis super content étais etait" # 

In [50]:
remove_stop_words(text)

"coucou c'est les amis super content etait "

## Most influential words

In [328]:
X_civile = formatted_X.where(y=='CIVILE')
X_commerciale = formatted_X.where(y=='COMMERCIALE')
X_criminelle = formatted_X.where(y=='CRIMINELLE')
X_sociale = formatted_X.where(y=='SOCIALE')

#to remove NaN values
X_civile = X_civile[~X_civile.isnull()]
X_commerciale = X_commerciale[~X_commerciale.isnull()]
X_criminelle = X_criminelle[~X_criminelle.isnull()]
X_sociale = X_sociale[~X_sociale.isnull()]

In [329]:
#so they all have differents shapes
print(X_civile.shape[0])
print(X_commerciale.shape[0])
print(X_criminelle.shape[0])
print(X_sociale.shape[0])

233
91
58
118


In [330]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def get_tfidf_matrix(X) : 
    
    count_vect = CountVectorizer()
    freq_term_matrix = count_vect.fit_transform(X)

    tfidf = TfidfTransformer(norm="l2")
    doc_tfidf_matrix = tfidf.fit_transform(freq_term_matrix)
    
    return (doc_tfidf_matrix, count_vect)

In [331]:
def get_influential_words(X, top=10) :
    
    tfidf_matrix, vec = get_tfidf_matrix(X)
    
    tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]
    feature_array = np.array(vec.get_feature_names())
    top_n = feature_array[tfidf_sorting][:top]
    
    return top_n

In [332]:
top_civile = get_influential_words(X_civile)
top_commerciale = get_influential_words(X_commerciale)
top_criminelle = get_influential_words(X_criminelle)
top_sociale = get_influential_words(X_sociale)

In [333]:
per_division = {'civile': top_civile, 'commerciale': top_commerciale, 'criminelle': top_criminelle, 'sociale': top_sociale}
words_df = pd.DataFrame(data = per_division)

words_df

,civile,commerciale,criminelle,sociale
0,accident,donneur,permis,reglement
1,salarié,ordre,conduire,delpointe
2,témoin,société,suspension,societe
3,travail,habib,opposition,52
4,2008,documentaire,arrêt,miniere
5,législation,bank,restituer,secours
6,employeur,transporteur,claude,coefficient
7,professionnelle,émettrice,véhicule,echelle
8,survenu,bénéficiaire,défaut,organigramme
9,août,crédit,1989,declassement


## Build document summarizer

Once you'll build the tf-idf matrix for the text corpus, you will use the tf-idf of each word to compute a value for each sentence. The n top sentences for a document will be used to represent the document.

In [334]:
def sent_tokenizer(text) : 
    return text.split('.')

In [335]:
formatted_sentences_X = formatted_X
formatted_sentences_X = formatted_sentences_X.apply(sent_tokenizer)
X = X.apply(sent_tokenizer)

In [336]:
#if fast_execution was chosen, find the 1st right index of X (because we only took a sample)

index = 0
while (index not in X) :
    index += 1
print(index)

116


In [337]:
print(formatted_sentences_X[index])
#formatted_X[i]
#X[index]

['cour cassation première chambre civile a rendu arrêt suivant joint les pourvois w 15-27 ', ' 127 v 15-27 ', ' 839 connexes ', ' recevabilité intervention associations union fédérale consommateurs-que choisir assurance emprunteur citoyen contestée société banque cic nord ouest attendu mémoires déposés greffe respectivement les 21 mars 11 avril 2017 les associations union fédérale consommateurs-que choisir assurance emprunteur citoyen déclaré intervenir appui prétentions mme x ', ' attendu interventions survenues avant ouverture débats recevables ', ' les moyens uniques pourvois réunis respectivement pris leurs première seconde branches vu les articles ', ' 113-12 code ', ' 312-9 code dernier rédaction antérieure celle issue loi 2010-737 1er juillet 2010 celle issue cette loi applicables cause ', ' attendu premier textes prévoit profit tant assuré assureur droit résilier contrat assurance moins deux mois avant date échéance annuelle ', ' vertu second droit ouvert cas contrat assurance 

In [338]:
tfidf_matrix, vec = get_tfidf_matrix(formatted_sentences_X[index])

In [339]:
def get_top_sentences_for_document(formatted_X, real_X, index, top=5) :

    top_sentences = []
    sentence_score = []
    tfidf_matrix, vec = get_tfidf_matrix(formatted_sentences_X[index])

    for i in range(tfidf_matrix.shape[0]) :
        mat = tfidf_matrix[i].toarray()

        # The score of the sentence corresponds to the sum of the tf-idf of each word / number of words
        sentence_score.append(np.sum(mat) / len(mat[mat > 0]))

    print(sentence_score) 
    top_sentence_score = np.argsort(sentence_score)[::-1][:top]
    print(top_sentence_score)

    for i in range(top) : 
        top_sentences.append(real_X[index][top_sentence_score[i]])

    return top_sentences

In [340]:
get_top_sentences_for_document(formatted_sentences_X, X, index)
q

[0.271550201963919, 0.5753768546264781, 0.7064917857822048, 0.16270766842236783, 0.3762381194577115, 0.27628223601711716, 0.573515230327525, 0.2412273247096299, 0.22450407550903115, 0.20335544562591581, 0.17069082784517775, 0.2039965556780803, 0.1480412296947794, 0.2518049408404454, 0.2352303466093587, 0.30647118903363857, 0.2874556613898631, 0.3431228627091546, 0.22340887540597346, 0.21243433722735716, 0.21098587512818653, 0.560605345625793, 0.3157275213081542, 0.23650451577995799, 0.2281503901615379, 0.33141119590475043, 0.29882299349461705, 0.16689923598301393, 0.5533775307273002, 0.2976617369710222, 0.33304631373287585, 0.2310421191505029, 0.3880302054804849, 0.24824518033190873, 0.2887576022023017, 0.3459475719764492, 0.36328991009526723, 0.15355132910270872, 0.2682867299546486, 0.573515230327525, 0.5661472248632763, 0.16906764547482675, 0.573515230327525, 0.3243775451453753, 0.14218704110521838, 0.573515230327525, 0.38471698809256066, 0.18720603964029703, 0.197851621733481, 0.293

/home/jerem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


['',
 '  ',
 ' 312-8 et L',
 ' 839 qui sont connexes ',
 ' 312-9 du code de la consommation ']

## STEP 3 : Split Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.2 ,random_state=42,stratify=df["division"] )
print(X_train.head())

## STEP 5 : Train the model

In [21]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [22]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

ValueError: could not convert string to float: "\nSur le moyen unique du pourvoi incident, qui est préalable :\n   Attendu, selon l'arrêt attaqué qu'un tribunal de grande instance a prononcé la séparation de corps des époux X... aux torts de l'épouse ; que, par assignation du 21 juillet 1989, le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce ; que, par assignation en date du 2 août 1989, Mme X..., autorisée par une ordonnance de non-conciliation du 19 janvier 1989, a assigné son mari en divorce pour faute ; qu'après jonction de ces deux procédures, le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme, et, accueillant la demande en divorce de celle-ci, prononcé également le divorce à son profit ; qu'ensuite le Tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés ;\n   Attendu qu'il est fait grief à l'arrêt infirmatif attaqué d'avoir déclaré irrecevable la demande en 

In [153]:
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)

ada_clf= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=100,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)

gdb_clf = GradientBoostingClassifier(random_state=42, n_estimators=100)

voting_clf = VotingClassifier(
    estimators=[('rfc', rnd_clf), ('abc', ada_clf), ('gbr', gdb_clf)],
    voting='hard')

In [154]:
for clf in (rnd_clf, ada_clf, gdb_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

ValueError: could not convert string to float: 'sur le moyen unique du pourvoi incident qui est préalable    attendu selon l arrêt attaqué qu un tribunal de grande instance a prononcé la séparation de corps des époux x aux torts de l épouse  que par assignation du 21 juillet 1989 le mari a assigné sa femme aux fins de conversion de la séparation de corps en divorce  que par assignation en date du 2 août 1989 mme x autorisée par une ordonnance de nonconciliation du 19 janvier 1989 a assigné son mari en divorce pour faute  qu après jonction de ces deux procédures le tribunal de grande instance a prononcé la conversion de la séparation de corps en divorce aux torts de la femme et accueillant la demande en divorce de celleci prononcé également le divorce à son profit  qu ensuite le tribunal a constaté que les époux étaient ainsi divorcés aux torts partagés    attendu qu il est fait grief à l arrêt infirmatif attaqué d avoir déclaré irrecevable la demande en divorce de la femme alors que d 